# 🌍 Get Coordinates of Any Location (Python Script)

This Python script allows you to enter any location name (city, country, or landmark) and returns its **latitude** and **longitude** using the **OpenStreetMap Nominatim API** via the `geopy` library.

---

## 🧠 Features
- 🌐 Works for any location worldwide.  
- 🔑 No API key required.  
- ⚡ Fast and simple — ideal for integration into larger projects like *AirSona*.  
- 🐍 Fully written in Python.  

---

## 🧰 Requirements

Install the required library:
```bash
pip install geopy


In [1]:
from geopy.geocoders import Nominatim


def get_coordinates(location_name):
    try:
        geolocation = Nominatim(user_agent="AirSona Location Finder")
        location = geolocation.geocode(location_name)
        if location:
            print(f"\n📍 Location: {location.address}")
            print(f"Latitude: {location.latitude}")
            print(f"Longitude: {location.longitude}\n")
            return location.address, location.latitude, location.longitude
        else:
            print("\n Location not found. Please try again. \n")

    except Exception as e:
        print(f"\n Error: {e} \n")
        return None
            

location = {} # it contain the location information


# create a main function 
if __name__ == "__main__":
    place = input("Enter a location (or 'exit to quit): ").strip()
    location['Entered_Place'] = place
    if place.lower() == "exit":
        print("Exciting program. ")

    
    address, latitude, Longitude =  get_coordinates(place)

    location["Address"] = address
    location["Latitude"] = latitude
    location["Longitude"] = Longitude

    print(location)


📍 Location: Lucknow, Uttar Pradesh, 226027, India
Latitude: 26.8381
Longitude: 80.9346001

{'Entered_Place': 'Lucknow', 'Address': 'Lucknow, Uttar Pradesh, 226027, India', 'Latitude': 26.8381, 'Longitude': 80.9346001}


In [2]:
import pandas as pd
import requests

# WAQI API Key
WAQI_API_KEY = "6018efdc1790561add0047b8778d806ad249b567"

# Function to fetch AQI data
def fetch_aqi_real_time(country, city=None):
    keyword = f"{city}, {country}" if city else country
    url = f"https://api.waqi.info/search/?token={WAQI_API_KEY}&keyword={keyword}"
    
    try:
        response = requests.get(url)
        if response.status_code != 200:
            return None

        data = response.json()
        if "data" not in data:
            return None

        aqi_data = []
        for station in data["data"]:
            if "station" in station and "geo" in station["station"]:
                lat, lon = station["station"]["geo"]
                station_name = station["station"]["name"]
                station_id = station["uid"]
                
                # Get detailed AQI info
                details_url = f"https://api.waqi.info/feed/@{station_id}/?token={WAQI_API_KEY}"
                details_response = requests.get(details_url)
                if details_response.status_code != 200:
                    continue
                
                details_data = details_response.json()
                if "data" not in details_data:
                    continue
                
                aqi = details_data["data"].get("aqi", None)

                aqi_data.append({
                    "lat": lat,
                    "lon": lon,
                    "location": station_name,
                    "AQI": int(aqi) if str(aqi).isdigit() else None
                })

        df = pd.DataFrame(aqi_data)
        df["AQI"] = df["AQI"].fillna(df["AQI"].mean())  # Fill missing AQI
        return df if not df.empty else None

    except Exception as e:
        print(f"Error: {e}")
        return None

# Main
if __name__ == "__main__":
    country = location['Entered_Place']
    city_or_state = location["Address"]

    print(f"\n🚀 Fetching real-time AQI data for {city_or_state}, {country}...\n")  
    aqi_data = fetch_aqi_real_time(country, city_or_state)



    Station_summaries = []


    if aqi_data is not None and not aqi_data.empty:
        print(f"🌍 Country: {country}")
        print(f"🏙 State/City: {city_or_state}\n")
        print("📌 AQI Monitoring Stations:")
        print("="*50)
        for _, row in aqi_data.iterrows():

            entry = (
                row['location'],
                row['lat'],
                row['lon']
            )
            Station_summaries.append(entry)

            location['AQI_Stations_data'] = {
                
            }

            print(f"📍 Station: {row['location']}")
            print(f"   ➤ Lat, Lon: ({row['lat']:.4f}, {row['lon']:.4f})")
            print("-"*50)
    else:
        print("❌ No AQI data found for the given location.")



🚀 Fetching real-time AQI data for Lucknow, Uttar Pradesh, 226027, India, Lucknow...

🌍 Country: Lucknow
🏙 State/City: Lucknow, Uttar Pradesh, 226027, India

📌 AQI Monitoring Stations:
📍 Station: Talkatora, Lucknow, India
   ➤ Lat, Lon: (26.8340, 80.8917)
--------------------------------------------------
📍 Station: Lalbagh, Lucknow, India
   ➤ Lat, Lon: (26.8458, 80.9387)
--------------------------------------------------
📍 Station: Central School, Lucknow, India
   ➤ Lat, Lon: (26.8821, 80.9303)
--------------------------------------------------
📍 Station: Nishant Ganj, Lucknow, India
   ➤ Lat, Lon: (26.8714, 80.9571)
--------------------------------------------------
📍 Station: Lalbagh, West Lucknow, Lucknow, India
   ➤ Lat, Lon: (26.8459, 80.9366)
--------------------------------------------------
📍 Station: Gomti Nagar, Lucknow, India
   ➤ Lat, Lon: (26.8681, 81.0051)
--------------------------------------------------
📍 Station: KTHM College, Nashik, India
   ➤ Lat, Lon: (19.9975,

NOW HERE, COORDINATES AND STATION NAME - WAQI API
STATIONS POLLUTANT DATA - OPENWEATHERAQI

In [3]:
# CELL 1 - imports & config
import requests
import json
from time import sleep

# Replace these with your real API keys
WAQI_TOKEN = "6018efdc1790561add0047b8778d806ad249b567"
OPENWEATHER_KEY = "103078b9eb141ee610be34a4c24288d8"

# small delay between calls to be polite to APIs
API_SLEEP = 0.2


In [4]:
# CELL 2 - search WAQI for stations matching a keyword/place
def waqi_search_stations(keyword, token=WAQI_TOKEN):
    """
    Uses WAQI Search API:
    GET https://api.waqi.info/search/?token={token}&keyword={keyword}
    Returns list of station dicts with at least: name, geo (lat, lon), uid, url, maybe aqi
    """
    base = "https://api.waqi.info/search/"
    params = {"token": token, "keyword": keyword}
    try:
        r = requests.get(base, params=params, timeout=10)
        data = r.json()
    except Exception as e:
        return {"status": "error", "message": f"WAQI search request failed: {e}"}
    
    if data.get("status") != "ok":
        return {"status": "error", "message": data.get("data", "unknown error from WAQI")}
    
    results = []
    for item in data.get("data", []):
        # typical returned structure (see WAQI docs / demo)
        station = item.get("station", {})
        geo = station.get("geo") or item.get("geo") or []
        lat = geo[0] if len(geo) > 0 else None
        lon = geo[1] if len(geo) > 1 else None
        results.append({
            "uid": item.get("uid"),
            "waqi_aqi": item.get("aqi"),
            "station_name": station.get("name") or item.get("station"),
            "station_url": station.get("url"),
            "lat": lat,
            "lon": lon
        })
    return {"status": "ok", "data": results}


In [5]:
# CELL — AQI formula implementation

def calculate_aqi_using_formula(components):
    """
    Calculates AQI from pollutant concentrations using US EPA breakpoints.
    Works for PM2.5, PM10, O3, CO, NO2, SO2.
    AQI = max(sub-index values for each pollutant)
    """

    # Breakpoints (EPA standard)
    breakpoints = {
        "pm2_5": [
            (0.0, 12.0, 0, 50),
            (12.1, 35.4, 51, 100),
            (35.5, 55.4, 101, 150),
            (55.5, 150.4, 151, 200),
            (150.5, 250.4, 201, 300),
            (250.5, 500.4, 301, 500),
        ],
        "pm10": [
            (0, 54, 0, 50),
            (55, 154, 51, 100),
            (155, 254, 101, 150),
            (255, 354, 151, 200),
            (355, 424, 201, 300),
            (425, 604, 301, 500),
        ],
        "o3": [
            (0.000, 0.054, 0, 50),
            (0.055, 0.070, 51, 100),
            (0.071, 0.085, 101, 150),
            (0.086, 0.105, 151, 200),
            (0.106, 0.200, 201, 300),
        ],
        "co": [
            (0.0, 4.4, 0, 50),
            (4.5, 9.4, 51, 100),
            (9.5, 12.4, 101, 150),
            (12.5, 15.4, 151, 200),
            (15.5, 30.4, 201, 300),
        ],
        "so2": [
            (0, 35, 0, 50),
            (36, 75, 51, 100),
            (76, 185, 101, 150),
            (186, 304, 151, 200),
            (305, 604, 201, 300),
        ],
        "no2": [
            (0, 53, 0, 50),
            (54, 100, 51, 100),
            (101, 360, 101, 150),
            (361, 649, 151, 200),
            (650, 1249, 201, 300),
        ]
    }

    def compute_subindex(cp, bplist):
        """Compute the sub-index using linear AQI formula."""
        for (low_c, high_c, low_i, high_i) in bplist:
            if low_c <= cp <= high_c:
                return round(((high_i - low_i) / (high_c - low_c)) * (cp - low_c) + low_i)
        return None  # out of range

    sub_indexes = []

    for pollutant, bp_list in breakpoints.items():
        cp = components.get(pollutant)
        if cp is not None:
            si = compute_subindex(cp, bp_list)
            if si is not None:
                sub_indexes.append(si)

    if not sub_indexes:
        return None

    return max(sub_indexes)  # AQI = highest sub-index


In [6]:
# CELL 3 - OpenWeather Air Pollution for coordinates
def openweather_pollutants(lat, lon, apikey=OPENWEATHER_KEY):
    """
    Calls OpenWeather Air Pollution API:
    GET http://api.openweathermap.org/data/2.5/air_pollution?lat={lat}&lon={lon}&appid={key}
    Returns dict with 'aqi' and 'components' if present
    """
    base = "http://api.openweathermap.org/data/2.5/air_pollution"
    params = {"lat": lat, "lon": lon, "appid": apikey}
    try:
        r = requests.get(base, params=params, timeout=10)
        data = r.json()
    except Exception as e:
        return {"status": "error", "message": f"OpenWeather request failed: {e}"}
    
    if "list" not in data or len(data["list"]) == 0:
        return {"status": "error", "message": "No air pollution data returned"}
    
    entry = data["list"][0]
    # map components safely (some fields might be missing)
    components = entry.get("components", {})

    calculate_aqi = calculate_aqi_using_formula(components)

    pollutants = {
        "co": components.get("co"),
        "no2": components.get("no2"),
        "o3": components.get("o3"),
        "so2": components.get("so2"),
        "pm2_5": components.get("pm2_5"),
        "pm10": components.get("pm10"),
        "AQI": calculate_aqi
    }
    return {"status": "ok", "data": {"ow_aqi": entry.get("main", {}).get("aqi"), "pollutants": pollutants}}


In [7]:
import json

with open("Tree_data.json", "r") as f:
    TREE_DATA = json.load(f)


In [14]:
def recommend_trees_from_dataset(pollutants, top_n=5):
    scored = []

    for tree in TREE_DATA:
        # ✅ FIX: Weight the tree's absorption rate by the local pollutant concentration.
        # Use .get(key, 0) to safely handle any missing pollutant data from the station.
        score = (
            tree.get("PM2_5_g_year", 0) * pollutants.get("pm2_5", 0) +
            tree.get("PM10_g_year", 0) * pollutants.get("pm10", 0) +
            tree.get("NO2_g_year", 0) * pollutants.get("no2", 0) +
            tree.get("SO2_g_year", 0) * pollutants.get("so2", 0) +
            tree.get("CO_g_year", 0) * pollutants.get("co", 0) +
            tree.get("O3_g_year", 0) * pollutants.get("o3", 0)
        )
        scored.append((tree, score))

    # Handle the case where all scores might be 0 (e.g., if pollutant data is missing/zero)
    # The sort and slice logic remains the same.
    scored.sort(key=lambda x: x[1], reverse=True)
    return scored[:top_n]


In [15]:
def canopy_area_from_density(density):
    density = density.lower()
    if "very" in density:
        return 70
    if "dense" in density:
        return 50
    if "medium" in density:
        return 35
    return 20  # light


In [16]:
def calculate_tree_pollutant_reduction_fixed(pollutants, tree, num_trees):
    """
    pollutants = openweather pollutant dict (µg/m³)
    tree       = tree data row containing PM2_5_g_year etc.
    num_trees  = 50 / 100 / 200
    """

    # Convert grams per year → micrograms per hour
    def g_year_to_ug_hour(g):
        return (g * 1_000_000) / (365 * 24)

    # canopy area per tree based on density
    canopy_area = canopy_area_from_density(tree["Canopy_Density"])
    total_area = canopy_area * num_trees

    # WHO urban mixing height
    mixing_height = 50  # meters
    air_volume_m3 = total_area * mixing_height

    reduction = {}
    reduction_list = []

    pollutant_keys = {
        "pm2_5": "PM2_5_g_year",
        "pm10": "PM10_g_year",
        "no2":  "NO2_g_year",
        "so2":  "SO2_g_year",
        "co":   "CO_g_year",
        "o3":   "O3_g_year"
    }

    for pol, tree_key in pollutant_keys.items():
        if pol not in pollutants:
            continue
        
        conc_ug_m3 = pollutants[pol]          # µg/m³
        air_mass_ug = conc_ug_m3 * air_volume_m3  # total µg in mixed air layer
        
        # tree absorption (µg/hour)
        tree_abs_ug_hour = g_year_to_ug_hour(tree[tree_key])

        # total tree absorption
        total_abs = tree_abs_ug_hour * num_trees

        # percent reduction
        percent = (total_abs / (air_mass_ug + 1e-12)) * 100

        # prevent unrealistic values
        percent = min(percent, 90)

        reduction[pol] = round(percent, 4)
        reduction_list.append(percent)

    # Average reduction
    avg_red = sum(reduction_list) / len(reduction_list)
    reduction["avg_reduction_percent"] = round(avg_red, 2)

    # New AQI
    original_aqi = pollutants["AQI"]
    new_aqi = original_aqi * (1 - avg_red / 100)

    reduction["new_AQI"] = max(round(new_aqi, 1), 0)

    return reduction


In [17]:
# CELL 4 - main aggregator (FULLY FIXED)
def get_place_stations_with_pollutants(place, max_stations=30):
    """
    1. Use WAQI search to get station candidates for `place`
    2. For each station (with lat/lon) call OpenWeather for pollutant data
    3. For each station generate tree recommendations
    4. Return combined JSON structure
    """
    out = {
        "query": place,
        "waqi_search": None,
        "stations": []
    }
    
    # 1) WAQI search
    waqi_res = waqi_search_stations(place)
    out["waqi_search"] = waqi_res
    
    if waqi_res.get("status") != "ok":
        return out  # return error info
    
    stations = waqi_res.get("data", [])
    if len(stations) == 0:
        return out
    
    stations = stations[:max_stations]
    
    # 2) Loop through stations
    for st in stations:
        lat, lon = st.get("lat"), st.get("lon")
        
        station_entry = {
            "uid": st.get("uid"),
            "station_name": st.get("station_name"),
            "station_url": st.get("station_url"),
            "waqi_aqi": st.get("waqi_aqi"),
            "lat": lat,
            "lon": lon,
            "openweather_status": None,
            "openweather_pollutants": None,
            "tree_recommendations": []  # FIXED: added correct field
        }
        
        # 3) Get OpenWeather pollutants
        if lat is not None and lon is not None:
            ow = openweather_pollutants(lat, lon)
            station_entry["openweather_status"] = ow.get("status")
            
            if ow.get("status") == "ok":
                pollutants = ow.get("data").get("pollutants", {})
                station_entry["openweather_pollutants"] = ow.get("data")
                
                # 4) Generate tree recommendations based on REAL pollutant data
                recommended = recommend_trees_from_dataset(pollutants)
                
                # 5) Add reduction calculations for 50, 100, 200 trees
                for tree_obj, score in recommended:
                    station_entry["tree_recommendations"].append({
                        "tree_name": tree_obj["Common_Name"],
                        "scientific_name": tree_obj["Scientific_Name"],
                        "canopy_density": tree_obj["Canopy_Density"],
                        "score": score,
                        "benefit_50": calculate_tree_pollutant_reduction_fixed(pollutants, tree_obj, 50),
                        "benefit_100": calculate_tree_pollutant_reduction_fixed(pollutants, tree_obj, 100),
                        "benefit_200": calculate_tree_pollutant_reduction_fixed(pollutants, tree_obj, 200),
                    })
            
            else:
                station_entry["openweather_pollutants"] = {"error": ow.get("message")}
            
            sleep(API_SLEEP)
        
        else:
            station_entry["openweather_status"] = "missing_coordinates"
        
        # 6) Store station
        out["stations"].append(station_entry)
    
    return out 

In [12]:
print(TREE_DATA)

[{'Tree_ID': 1, 'Common_Name': 'Neem', 'Scientific_Name': 'Azadirachta indica', 'Leaf_Type': 'Deciduous', 'Canopy_Density': 'Dense', 'Native_Region': 'All India (Tropical)', 'PM2_5_g_year': 220.5, 'PM10_g_year': 750.8, 'NO2_g_year': 110.4, 'SO2_g_year': 70.2, 'CO_g_year': 3200.7, 'O3_g_year': 1100.3}, {'Tree_ID': 2, 'Common_Name': 'Peepal', 'Scientific_Name': 'Ficus religiosa', 'Leaf_Type': 'Deciduous', 'Canopy_Density': 'Very Dense', 'Native_Region': 'All India', 'PM2_5_g_year': 250.2, 'PM10_g_year': 800.6, 'NO2_g_year': 120.1, 'SO2_g_year': 80.4, 'CO_g_year': 3400.8, 'O3_g_year': 1200.5}, {'Tree_ID': 3, 'Common_Name': 'Banyan', 'Scientific_Name': 'Ficus benghalensis', 'Leaf_Type': 'Evergreen', 'Canopy_Density': 'Very Dense', 'Native_Region': 'All India', 'PM2_5_g_year': 300.1, 'PM10_g_year': 900.9, 'NO2_g_year': 150.3, 'SO2_g_year': 90.2, 'CO_g_year': 3500.4, 'O3_g_year': 1300.9}, {'Tree_ID': 4, 'Common_Name': 'Mango', 'Scientific_Name': 'Mangifera indica', 'Leaf_Type': 'Evergreen', 

In [18]:
# CELL 5 - run interactive
place = input("Enter place / city / keyword to search WAQI stations: ").strip()
if not place:
    print("Please enter a place.")
else:
    result = get_place_stations_with_pollutants(place, max_stations=40)
    print(json.dumps(result, indent=2, ensure_ascii=False))


{
  "query": "Lucknow",
  "waqi_search": {
    "status": "ok",
    "data": [
      {
        "uid": 8673,
        "waqi_aqi": "189",
        "station_name": "Lalbagh, West Lucknow, Lucknow, India",
        "station_url": "india/lucknow/lalbagh--west-lucknow",
        "lat": 26.8458805,
        "lon": 80.9365541
      },
      {
        "uid": 8188,
        "waqi_aqi": "164",
        "station_name": "Talkatora, Lucknow, India",
        "station_url": "india/lucknow/talkatora",
        "lat": 26.83399722,
        "lon": 80.8917361
      },
      {
        "uid": 3845,
        "waqi_aqi": "158",
        "station_name": "Central School, Lucknow, India",
        "station_url": "india/lucknow/central-school",
        "lat": 26.8821003,
        "lon": 80.9302753
      },
      {
        "uid": 13721,
        "waqi_aqi": "158",
        "station_name": "B R Ambedkar University, Lucknow, India",
        "station_url": "india/lucknow/b-r-ambedkar-university",
        "lat": 26.766433,
        "lo